In [41]:
import numpy as np

In [55]:
def polynom_matrix(a_min,a_max,b_min,b_max,seq_length):
    
    a_array = np.arange(a_min,a_max+1)
    b_array = np.arange(b_min,b_max+1)
    
    u_array = np.arange(seq_length)
    v_array = u_array
    
    au_array = np.multiply.outer(a_array,u_array**2)
    buv_array = np.multiply.outer(b_array,np.multiply.outer(u_array,v_array))
    
    corrected_au_array = np.broadcast_to(au_array,(a_max+1-a_min,b_max+1-b_min,seq_length,seq_length))
    corrected_buv_array = np.broadcast_to(buv_array,(a_max+1-a_min,b_max+1-b_min,seq_length,seq_length))
    
    Poly_matrix = corrected_au_array + corrected_buv_array # Some numpy trick
    
    return Poly_matrix

In [60]:
polynom_matrix(-10,10,-10,10,21)

array([[[[    0,   -10,   -40, ..., -3240, -3610, -4000],
         [    0,   -19,   -56, ..., -3096, -3439, -3800],
         [    0,   -28,   -72, ..., -2952, -3268, -3600],
         ...,
         [    0,  -172,  -328, ...,  -648,  -532,  -400],
         [    0,  -181,  -344, ...,  -504,  -361,  -200],
         [    0,  -190,  -360, ...,  -360,  -190,     0]],

        [[    0,   -10,   -40, ..., -3240, -3610, -4000],
         [    0,   -18,   -54, ..., -3078, -3420, -3780],
         [    0,   -26,   -68, ..., -2916, -3230, -3560],
         ...,
         [    0,  -154,  -292, ...,  -324,  -190,   -40],
         [    0,  -162,  -306, ...,  -162,     0,   180],
         [    0,  -170,  -320, ...,     0,   190,   400]],

        [[    0,   -10,   -40, ..., -3240, -3610, -4000],
         [    0,   -17,   -52, ..., -3060, -3401, -3760],
         [    0,   -24,   -64, ..., -2880, -3192, -3520],
         ...,
         [    0,  -136,  -256, ...,     0,   152,   320],
         [    0,  -143,  -

In [ ]:
%time